In [ ]:
# Facial Emotion Recognition using CNN (CSV-Based)
# Dataset: FER-2013 (48x48 grayscale images stored as pixel values in CSV)
# Classes: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral
# Source: https://www.kaggle.com/datasets/deadskull7/fer2013

In [ ]:
# Import required libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Check GPU availability (optional)
#print("GPU Available:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set up Kaggle token and download the dataset
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle_token/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip FER-2013 dataset
!kaggle datasets download deadskull7/fer2013
!unzip -q fer2013.zip -d fer2013


In [ ]:
# Load the FER-2013 CSV file
data = pd.read_csv('/content/fer2013/fer2013.csv')

# Display first few rows and class distribution
print(data.head())
print(data['emotion'].value_counts())


In [ ]:
# Display one sample image per emotion category

emotion_labels = {
    0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy",
    4: "Sad", 5: "Surprise", 6: "Neutral"
}

plt.figure(figsize=(14, 6))

for emotion in range(7):
    row = data[data['emotion'] == emotion].iloc[0]
    pixels = np.fromstring(row['pixels'], dtype=int, sep=' ')
    image = pixels.reshape(48, 48)

    plt.subplot(2, 4, emotion + 1)
    plt.imshow(image, cmap='gray')
    plt.title(emotion_labels[emotion])
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Data Preprocessing: prepare input features and labels

labels = data['emotion'].values
pixels = data['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
X = np.stack(pixels.values).reshape(-1, 48, 48, 1) / 255.0
y = to_categorical(labels, num_classes=7)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Label shape (train):", y_train.shape)


In [ ]:
# Build the CNN model

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.summary()


In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the CNN model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=64,
    verbose=1
)

In [ ]:
# Plot training and validation accuracy & loss

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Train Acc')
plt.plot(epochs, val_acc, 'ro-', label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Train Loss')
plt.plot(epochs, val_loss, 'ro-', label='Val Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Predict on validation set
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

# Classification report
print(classification_report(y_true, y_pred_classes))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=emotion_labels.values(),
            yticklabels=emotion_labels.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()